In [1]:
    if __name__ == '__main__': 
        %reload_ext pidgin

In [2]:
    from pidgin import shell, specification, formatter
    import builtins, string, jinja2, nbconvert, IPython, textwrap, typing as t, traitlets

In [3]:
    regex = ".*\{\{.* |.*\}\}.*"

In [4]:
    def show_axes(object):
>>> %matplotlib agg
>>> import pandas
>>> with IPython.utils.capture.capture_output() as out:
...     get_ipython().weave.run_cell("{"+"{pandas.DataFrame([1]).plot()}}")
>>> out.outputs[0].data
{'text/plain': '<IPython.core.display.HTML object>', 'text/html': '<p><img src="data:image/png;base64,..."/></p>\n'}

        import matplotlib.backends.backend_svg
        bytes = __import__('io').BytesIO()
        matplotlib.backends.backend_agg.FigureCanvasAgg(getattr(object, 'figure', object)).print_png(bytes)
        return format_bytes(bytes.getvalue(), object)

1,def show_axes(object):
9 10 11 12,"import matplotlib.backends.backend_svg bytes = __import__('io').BytesIO() matplotlib.backends.backend_agg.FigureCanvasAgg(getattr(object, 'figure', object)).print_png(bytes) return format_bytes(bytes.getvalue(), object)"


In [5]:
    def show_sympy_axes(object): 
>>> import sympy
>>> # with IPython.utils.capture.capture_output() as out:
... #    get_ipython().weave.run_cell("{"+"{sympy.plotting.plot(sympy.Symbol('x'), ('x', -10, 10))}}")
>>> # out.outputs[0].data
        
        s = __import__('io').BytesIO()
        object.save(s)
        return format_bytes(s.getvalue(), object)

1,def show_sympy_axes(object):
7 8 9,"s = __import__('io').BytesIO() object.save(s) return format_bytes(s.getvalue(), object)"


In [6]:
    class Template(nbconvert.exporters.TemplateExporter, formatter.Dollar):
        special_formatters = traitlets.List(default_value=(
                ('matplotlib.figure', 'Axes', show_axes),
                ('matplotlib.figure', 'Figure', show_axes),
                ('matplotlib.axes._subplots', 'AxesSubplot', show_axes),
                ('sympy.plotting.plot', 'Plot', show_sympy_axes),
            ))
        def __init__(Template, *args, **kwargs):
            formatter.Dollar.__init__(Template, *args, **kwargs)
            nbconvert.exporters.TemplateExporter.__init__(Template, *args, **kwargs)
            Template._create_environment()
            [Template.mimebundle_formatter.for_type_by_name(*object) for object in Template.special_formatters]
                        
        def default_filters(Template):
            yield from specification.colors.filters.items()
            yield from super().default_filters()
            
        def _create_environment(Template):
            environment = super()._create_environment()
            environment.undefined = jinja2.StrictUndefined
            environment.finalize = Template.finalize
            return environment
        
        def render(Template, object, **kwargs):
            ns = {**vars(builtins), **Template.user_ns, **kwargs}
            Template.environment.filters = {**{k: v for k, v in ns.items() if callable(v)}, **Template.environment.filters}
            object = super().render(object)
            return Template.environment.from_string(object).render(**ns)
        
        def finalize(Template, object) -> str:
            if isinstance(object, str): 
                new = Template.user_ns.get(object, object)
                if new == object:
                    return object
                if isinstance(new, str):
                    return Template.finalize(new)
                
            bundle, metadata = Template.format(object)
            for type in [str for str in reversed(Template.active_types) if str != 'text/plain']:
                if type in bundle: 
                    object = bundle[type]
                    if type.startswith('image') and ('svg' not in type): 
                        object = format_images(type, bundle)
                    if type == 'text/latex': 
                        if object.startswith('$$') and object.endswith('$$'): object = object[1:-1]
                    break
            return object

In [7]:
    def format_bytes(bytes, object):
        return {'text/html': format_images('image/png', {'image/png': bytes}), 'text/plain': repr(object),}, {}

In [8]:
    def format_images(type, bundle):
        str = bundle[type]        
        if isinstance(str, bytes): str = __import__('base64').b64encode(str).decode('utf-8')
        if type in ('image/svg+xml', 'text/html'):  ...
        elif str.startswith('http'): str = F"""<img src="{str}"/>"""
        else: str = F"""<img src="data:{type};base64,{str}"/>"""
        return str

In [9]:
    def event(shell, event): return list(shell.template.environment.filters) if hasattr(shell, 'template') else []

In [10]:
    def load_ipython_extension(shell: IPython.InteractiveShell):
        shell.set_hook('complete_command', event, re_key=regex)

In [11]:
    def unload_ipython_extension(shell):
        try:
            del shell.strdispatchers['complete_command'].regexs[regex]
        except KeyError: ...